In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer, SnowballStemmer, PorterStemmer
from utils import reorganize_target_char, extract_url, lowercase, clean_text, tokenize_text

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/antoine/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data_train = pd.read_csv('../data/train_tweets.csv')
data_test = pd.read_csv('../data/test_tweets.csv')

In [3]:
data_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
data_train = reorganize_target_char(data_train)
data_train.head(15)

,id,keyword,location,text,target,hashtags,mentions
0,1,NaN,NaN,Our Deeds are the Reason of this earthquake Ma...,1,earthquake,None
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,None,None
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,None,None
3,6,NaN,NaN,"13,000 people receive wildfires evacuation ord...",1,wildfires,None
4,7,NaN,NaN,Just got sent this photo from Ruby Alaska as s...,1,"Alaska, wildfires",None
5,8,NaN,NaN,RockyFire Update => California Hwy. 20 closed ...,1,"RockyFire, CAfire, wildfires",None
6,10,NaN,NaN,flood disaster Heavy rain causes flash floodin...,1,"flood, disaster",None
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1,None,None
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1,None,None
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1,None,None


In [5]:
data_train = extract_url(data_train)

In [6]:
data_train = lowercase(data_train)
data_train.head()

,id,keyword,location,text,target,hashtags,mentions,urls
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1,earthquake,None,None
1,4,NaN,NaN,forest fire near la ronge sask. canada,1,None,None,None
2,5,NaN,NaN,all residents asked to 'shelter in place' are ...,1,None,None,None
3,6,NaN,NaN,"13,000 people receive wildfires evacuation ord...",1,wildfires,None,None
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1,"alaska, wildfires",None,None


In [7]:
"Mentions : ",data_train['mentions'].notna().sum() ,"Hashtags : ",data_train['hashtags'].notna().sum(), "URLS", data_train['urls'].notna().sum()

('Mentions : ', 2009, 'Hashtags : ', 1743, 'URLS', 3971)

In [8]:
data_train[data_train['urls'].notna()].head()

,id,keyword,location,text,target,hashtags,mentions,urls
31,48,ablaze,birmingham,bbcmtd wholesale markets ablaze,1,None,bbcmtd,http://t.co/lhyxeohy6c
32,49,ablaze,est. september 2012 - bristol,we always try to bring the heavy. metal rt,0,"metal, rt",None,http://t.co/yao1e0xngw
33,50,ablaze,africa,africanbaze: breaking news:nigeria flag set ab...,1,africanbaze,None,http://t.co/2nndbgwyei
35,53,ablaze,"london, uk",on plus side look at the sky last night it was...,0,None,None,http://t.co/qqsmshaj3n
37,55,ablaze,world wide!!,inec office in abia set ablaze -,1,None,None,http://t.co/3imaomknna


In [9]:
data_train[data_train['mentions'].notna()].head()

,id,keyword,location,text,target,hashtags,mentions,urls
31,48,ablaze,birmingham,bbcmtd wholesale markets ablaze,1,None,bbcmtd,http://t.co/lhyxeohy6c
36,54,ablaze,pretoria,phdsquares mufc they've built so much hype aro...,0,mufc,phdsquares,None
43,63,ablaze,NaN,soooo pumped for ablaze ???? southridgelife,0,None,southridgelife,None
54,78,ablaze,abuja,noches el-bestia 'alexis_sanchez: happy to see...,0,None,alexis_sanchez,http://t.co/uc4j4jhvgr'
63,91,ablaze,"concord, ca",navista7 steve these fires out here are someth...,1,None,"navista7, news24680",None


In [10]:
data_train = clean_text(data_train)
data_train.head()

,id,keyword,location,text,target,hashtags,mentions,urls
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1,earthquake,None,None
1,4,NaN,NaN,forest fire near la ronge sask canada,1,None,None,None
2,5,NaN,NaN,all residents asked to shelter in place are ...,1,None,None,None
3,6,NaN,NaN,13 000 people receive wildfires evacuation ord...,1,wildfires,None,None
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1,"alaska, wildfires",None,None


In [12]:
data_train = tokenize_text(data_train)
data_train.head()

,id,keyword,location,text,target,hashtags,mentions,urls,text_tokenize
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1,earthquake,None,None,"[our, deeds, are, the, reason, of, this, earth..."
1,4,NaN,NaN,forest fire near la ronge sask canada,1,None,None,None,"[forest, fire, near, la, ronge, sask, canada]"
2,5,NaN,NaN,all residents asked to shelter in place are ...,1,None,None,None,"[all, residents, asked, to, shelter, in, place..."
3,6,NaN,NaN,13 000 people receive wildfires evacuation ord...,1,wildfires,None,None,"[13, 000, people, receive, wildfires, evacuati..."
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1,"alaska, wildfires",None,None,"[just, got, sent, this, photo, from, ruby, ala..."
